In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [3]:
df = pd.read_csv('/home/igor/Elbrus/Project-/release/data.csv')
df.columns

Index(['id', 'rooms', 'adress', 'area', 'building', 'parking', 'price',
       'description', 'repair', 'rooms_square', 'balcony', 'windows',
       'bathroom', 'сhildren/pets_allowed', 'additionally', 'housing_complex',
       'ceiling_height', 'elevator', 'garbage_chute', 'floor_count'],
      dtype='object')

In [4]:
my_df = df.iloc[:,:6]

In [5]:
my_df.head()

,id,rooms,adress,area,building,parking
0,271271157,4,"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная
1,271634126,4,"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная
2,271173086,"4, Оба варианта","Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная
3,272197456,"4, Оба варианта","Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная
4,273614615,2,"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",Unknown


## rooms
В столбце содержится информация о числе комнат и о планировке. Разобьем его на два признака: rooms_number и layout

In [6]:
my_df['rooms_number'] = my_df['rooms'].apply(lambda x: int(x.split(', ')[0]) if len(x) > 1 else int(x))

In [7]:
my_df['layout'] = my_df['rooms'].apply(lambda x: x.split(', ')[1] if len(x) > 1 else 'Unknown')

In [8]:
my_df['price'] = df['price'].apply(lambda x: int(x.split('.')[0]))

In [9]:
def set_currency(x):
    if '€' in x:
        return 'eur'
    elif '$' in x:
        return 'usd'
    elif 'руб' in x:
        return 'rub'
    else:
        return 'other'

In [10]:
my_df['currency'] = df['price'].apply(set_currency)

In [11]:
my_df.groupby('currency').size()

currency
eur        9
rub    19714
usd       14
dtype: int64

In [12]:
my_df.loc[my_df['currency'] == 'eur', 'price'] = (my_df.loc[my_df['currency'] == 'eur', 'price'] * 100.07).astype(int)
my_df.loc[my_df['currency'] == 'usd', 'price'] = (my_df.loc[my_df['currency'] == 'usd', 'price'] * 92.48).astype(int)

In [13]:
my_df['price'].describe()

count     19737
mean      87851
std      134420
min       12000
25%       40000
50%       50000
75%       75000
max     3000000
Name: price, dtype: float64

In [14]:
# Удалим ненужные столбцы

my_df = my_df.drop(columns=['rooms', 'currency'])

In [15]:
# Теперь можно проверить как связана планировка со стоимостью аренды

In [16]:
my_df.groupby('layout').agg({'price': ['count', 'median']})

price       
               count median
layout                     
Unknown        11770  45000
Изолированная   5803  55000
Оба варианта    1492  85000
Смежная          672  45000

In [17]:
# Применяем target encoding к категориальной переменной btype
median_price_per_layout = my_df.groupby('layout')['price'].median()

# Создадим словарь, сопоставляющий категории building_type с их медианными значениями цены
layout_median_price_map = median_price_per_layout.to_dict()

# Заменим значения в столбце building_type на их медианные значения цены
my_df['layout'] = my_df['layout'].map(layout_median_price_map)

## area

Вытащим из столбца общую площадь

In [18]:
my_df['area'] = my_df['area'].apply(lambda x: int(float(x.split('/')[0])) if len(x.split('/')) > 1 else int(float(x)))

In [19]:
my_df.head(1)

,id,adress,area,building,parking,rooms_number,layout,price
0,271271157,"Москва, улица Новый Арбат, 27",200,"5/16, Монолитный",подземная,4,45000,500000


## building

Разделим этот признак на 3: этаж, на котором находится квартира, сколько всего этажей в доме и какого типа дом

In [20]:
my_df['btype'] = my_df['building'].apply(lambda x: x.split(', ')[1] if len(x.split(', ')) > 1 else 'Unknown')

In [21]:
my_df.groupby('btype').size().sort_values(ascending=False)

btype
Панельный              6679
Кирпичный              3696
Монолитный             3615
Unknown                2971
Блочный                1689
Монолитно-кирпичный     872
Сталинский              141
старый фонд              68
Деревянный                5
Щитовой                   1
dtype: int64

In [22]:
my_df.groupby('btype').agg({'price': ['mean', 'median', 'count']})

price             
                      mean median count
btype                                  
Unknown             139597  57000  2971
Блочный              45096  40000  1689
Деревянный           79000  35000     5
Кирпичный            82055  55000  3696
Монолитно-кирпичный 198497 120000   872
Монолитный          116255  70000  3615
Панельный            46591  43000  6679
Сталинский          113957  95000   141
Щитовой              45000  45000     1
старый фонд         274853 210000    68

In [23]:
# Применяем target encoding к категориальной переменной btype
median_price_per_building_type = my_df.groupby('btype')['price'].median()

# Создадим словарь, сопоставляющий категории building_type с их медианными значениями цены
building_type_median_price_map = median_price_per_building_type.to_dict()

# Заменим значения в столбце building_type на их медианные значения цены
my_df['btype'] = my_df['btype'].map(building_type_median_price_map)

In [24]:
# Выделим этаж, на которой находится квартра и общую этажность дома в отдельные столбцы
my_df['floor'] = my_df['building'].apply(lambda x: int(x.split(', ')[0].split('/')[0]) if len(x.split(', ')) > 1 else int(x.split('/')[0]))
my_df['total_floors'] = my_df['building'].apply(lambda x: int(x.split(', ')[0].split('/')[1]) if len(x.split(', ')) > 1 else int(x.split('/')[1]))

In [25]:
my_df.drop(columns='building', inplace=True)
my_df.head()

,id,adress,area,parking,rooms_number,layout,price,btype,floor,total_floors
0,271271157,"Москва, улица Новый Арбат, 27",200,подземная,4,45000,500000,70000,5,16
1,271634126,"Москва, улица Новый Арбат, 27",198,подземная,4,45000,500000,120000,5,16
2,271173086,"Москва, улица Новый Арбат, 27",200,подземная,4,85000,500000,57000,5,16
3,272197456,"Москва, переулок Плотников, 21С1",170,подземная,4,85000,400000,57000,5,6
4,273614615,"Москва, улица Новый Арбат, 15",58,Unknown,2,45000,225000,43000,12,26


In [26]:
# Поделим total_floors на квантили
my_df['total_floors_quantile'] = pd.qcut(my_df['total_floors'], q=5, labels=False)

In [27]:
# Применяем target encoding к квантилям total_floors
mean_price_per_total_floors_quantile = my_df.groupby('total_floors_quantile')['price'].mean()

# Создадим словарь, сопоставляющий категории building_type с их медианными значениями цены
total_floors_quantile_mean_price_map = mean_price_per_total_floors_quantile.to_dict()

# Заменим значения в столбце building_type на их медианные значения цены
my_df['total_floors_quantile'] = my_df['total_floors_quantile'].map(total_floors_quantile_mean_price_map)

In [28]:
# Сделаем то же самое с этажностью здания
my_df['floor_quantile'] = pd.qcut(my_df['floor'], q=5, labels=False)

In [29]:
# Применяем target encoding к квантилям floor
mean_price_per_floor_quantile = my_df.groupby('floor_quantile')['price'].mean()

# Создадим словарь, сопоставляющий категории building_type с их медианными значениями цены
floor_quantile_mean_price_map = mean_price_per_floor_quantile.to_dict()

# Заменим значения в столбце building_type на их медианные значения цены
my_df['floor_quantile'] = my_df['floor_quantile'].map(floor_quantile_mean_price_map)

In [30]:
my_df.drop(columns=['floor', 'total_floors'], inplace=True)

In [31]:
my_df.head(1)

,id,adress,area,parking,rooms_number,layout,price,btype,total_floors_quantile,floor_quantile
0,271271157,"Москва, улица Новый Арбат, 27",200,подземная,4,45000,500000,70000,68621,95289


## Парковка
Сравним стоимость аренды в зависимости от наличия информации о парковке

In [32]:
my_df['has_parkin'] = my_df['parking'].apply(lambda x: 0 if x == 'Unknown' else 1)
my_df.groupby('has_parkin').agg({'price': ['mean', 'median', 'count']})

price              
            mean median  count
has_parkin                    
0          79463  45000  11174
1          98796  55000   8563

In [33]:
# Для обучения модели оставим бинарный признак, а изначальный столбец удалим

In [34]:
my_df.drop(columns=['parking', 'adress'], inplace=True)

In [35]:
my_df = my_df.set_index('id')

In [36]:
my_df.head(5)

,area,rooms_number,layout,price,btype,total_floors_quantile,floor_quantile,has_parkin
id,,,,,,,,
271271157,200,4,45000,500000,70000,68621,95289,1
271634126,198,4,45000,500000,120000,68621,95289,1
271173086,200,4,85000,500000,57000,68621,95289,1
272197456,170,4,85000,400000,57000,128076,95289,1
273614615,58,2,45000,225000,43000,104609,94374,0


In [37]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19737 entries, 271271157 to 274672243
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   area                   19737 non-null  int64  
 1   rooms_number           19737 non-null  int64  
 2   layout                 19737 non-null  float64
 3   price                  19737 non-null  int64  
 4   btype                  19737 non-null  float64
 5   total_floors_quantile  19737 non-null  float64
 6   floor_quantile         19737 non-null  float64
 7   has_parkin             19737 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 1.4 MB


In [38]:
df.to_csv('igor.csv', index=False)

In [40]:
# Возвращаею потерявшийся столбец
df_metro = pd.read_csv('/home/igor/Elbrus/Project-/tmp_data/_data (1).csv')
df_metro = df_metro.set_index('ID  объявления')

In [41]:
my_df['metro'] = df_metro['Метро']

In [42]:
my_df['metro'] = my_df['metro'].apply(lambda x: x.split(' (')[0] if isinstance(x, str) else 'Unknown')

In [43]:
my_df.groupby('metro').size().sort_values(ascending=False)

metro
м. Селигерская     393
Unknown            346
м. Щелковская      313
м. Бабушкинская    284
м. Новогиреево     282
                  ... 
м. Театральная       1
м. Лужники           1
м. Румянцево         1
м. Нахабино          1
м. Яхромская         1
Length: 278, dtype: int64

In [44]:
# Применяем target encoding к metro
mean_price_per_metro = my_df.groupby('metro')['price'].mean()

# Создадим словарь, сопоставляющий категории metro с их средними значениями цены
metro_mean_price_map = mean_price_per_metro.to_dict()

# Заменим значения в столбце metro на их средние значения цены
my_df['metro'] = my_df['metro'].map(metro_mean_price_map)

In [45]:
my_df

,area,rooms_number,layout,price,btype,total_floors_quantile,floor_quantile,has_parkin,metro
id,,,,,,,,,
271271157,200,4,45000,500000,70000,68621,95289,1,313197
271634126,198,4,45000,500000,120000,68621,95289,1,313197
271173086,200,4,85000,500000,57000,68621,95289,1,313197
272197456,170,4,85000,400000,57000,128076,95289,1,313197
273614615,58,2,45000,225000,43000,104609,94374,0,340904
...,...,...,...,...,...,...,...,...,...
215565511,35,1,45000,42000,57000,68621,72970,0,57379
274654844,38,1,45000,45000,70000,64310,95289,0,46280
268679909,43,2,85000,50000,55000,128076,95289,0,46280


In [47]:
df.to_csv('../release/data.csv', index=False)